In [1]:
pip install -q peft

Note: you may need to restart the kernel to use updated packages.


In [2]:
!pip install -q bitsandbytes

In [3]:
!pip install -q trl

In [4]:
!pip install sentencepiece

In [5]:
!pip install -q -U einops

In [77]:
!pip install --upgrade trl

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [6]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("saadmakhdoom/ecommerce-faq-chatbot-dataset")

print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/ecommerce-faq-chatbot-dataset


In [99]:
import json
import os
from pprint import pprint
import bitsandbytes as bnb
import pandas as pd
import torch
import torch.nn as nn
import transformers
from datasets import load_dataset
from huggingface_hub import notebook_login
from peft import (
LoraConfig,
PeftConfig,
PeftModel,
get_peft_model,
prepare_model_for_kbit_training,
)
from transformers import (
AutoConfig,
AutoModelForCausalLM,
AutoTokenizer,
BitsAndBytesConfig
)
from transformers import DataCollatorWithPadding
from trl import SFTTrainer
from transformers.models.falcon.modeling_falcon import FalconForCausalLM

In [101]:
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

In [102]:
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

In [103]:
with open('/kaggle/input/ecommerce-faq-chatbot-dataset/Ecommerce_FAQ_Chatbot_dataset.json') as json_file:
    data = json.load(json_file)

In [104]:
pprint(data['questions'][0], sort_dicts = False)

{'question': 'How can I create an account?',
 'answer': "To create an account, click on the 'Sign Up' button on the top "
           'right corner of our website and follow the instructions to '
           'complete the registration process.'}


In [105]:
pprint(data['questions'][1], sort_dicts = False)

{'question': 'What payment methods do you accept?',
 'answer': 'We accept major credit cards, debit cards, and PayPal as payment '
           'methods for online orders.'}


In [106]:
pprint(data['questions'][2], sort_dicts = False)

{'question': 'How can I track my order?',
 'answer': 'You can track your order by logging into your account and '
           "navigating to the 'Order History' section. There, you will find "
           'the tracking information for your shipment.'}


In [107]:
with open('dataset.json', 'w') as f:
    json.dump(data['questions'], f)

In [108]:
pd.DataFrame(data['questions'])

,question,answer
0,How can I create an account?,"To create an account, click on the 'Sign Up' b..."
1,What payment methods do you accept?,"We accept major credit cards, debit cards, and..."
2,How can I track my order?,You can track your order by logging into your ...
3,What is your return policy?,Our return policy allows you to return product...
4,Can I cancel my order?,You can cancel your order if it has not been s...
...,...,...
74,Can I order a product if it is listed as 'sold...,If a product is listed as 'sold out' but avail...
75,Can I return a product if it was purchased wit...,"Yes, you can return a product purchased with a..."
76,Can I request a product if it is not currently...,If a product is not available in your preferre...
77,Can I order a product if it is listed as 'comi...,If a product is listed as 'coming soon' but no...


In [109]:
MODEL_NAME = 'tiiuae/falcon-7b'

In [110]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit = True,
    bnb_4bit_use_double_quant = True,
    bnb_4bit_quant_type = 'nf4',
    bnb_4bit_compute_dtype = torch.float16
)

In [111]:
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    device_map = 'auto',
    trust_remote_code = True,
    quantization_config = bnb_config,
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [112]:
def patched_forward(self, *args, **kwargs):
    allowed_keys = {
        'input_ids', 'attention_mask', 'position_ids', 'past_key_values',
        'token_type_ids', 'inputs_embeds', 'labels', 'use_cache',
        'output_attentions', 'output_hidden_states', 'return_dict'
    }
    kwargs = {k: v for k, v in kwargs.items() if k in allowed_keys}
    return self.__original_forward__(*args, **kwargs)

In [113]:
if not hasattr(FalconForCausalLM, '__original_forward__'):
    FalconForCausalLM.__original_forward__ = FalconForCausalLM.forward
    FalconForCausalLM.forward = patched_forward

In [114]:
torch.cuda.empty_cache()

In [115]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
tokenizer.pad_token = tokenizer.eos_token

In [116]:
def print_trainable_parameters(model):
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(f'trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}')

In [117]:
model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

You are using an old version of the checkpointing format that is deprecated (We will also silently ignore `gradient_checkpointing_kwargs` in case you passed it).Please update to the new format on your modeling file. To use the new format, you need to completely remove the definition of the method `_set_gradient_checkpointing` in your model.
You are using an old version of the checkpointing format that is deprecated (We will also silently ignore `gradient_checkpointing_kwargs` in case you passed it).Please update to the new format on your modeling file. To use the new format, you need to completely remove the definition of the method `_set_gradient_checkpointing` in your model.


In [118]:
config = LoraConfig(
    r = 16,
    lora_alpha = 32,
    target_modules = ['query_key_value'],
    lora_dropout = 0.05,
    bias = 'none',
    task_type = 'CAUSAL_LM',
)

In [119]:
model = get_peft_model(model, config)
print_trainable_parameters(model)

trainable params: 4718592 || all params: 3613463424 || trainable%: 0.13058363808693696


In [120]:
prompt = f"""
<human>: How can I create an account?
<assistant>:
""".strip()
print(prompt)

<human>: How can I create an account?
<assistant>:


In [121]:
generation_config = model.generation_config
generation_config.max_new_tokens = 200
generation_config.temperature = 0.7
generation_config.top_p = 0.7
generation_config.num_return_sequences = 1
generation_config.pad_token_id = tokenizer.eos_token_id
generation_config.eos_token_id = tokenizer.eos_token_id

In [122]:
generation_config

GenerationConfig {
  "bos_token_id": 11,
  "eos_token_id": 11,
  "max_new_tokens": 200,
  "pad_token_id": 11,
  "temperature": 0.7,
  "top_p": 0.7
}

In [123]:
device = 'cuda:0'
encoding = tokenizer(prompt, return_tensors = 'pt').to(device)
with torch.inference_mode():
    outputs = model.generate(
        input_ids = encoding.input_ids,
        attention_mask = encoding.attention_mask,
        generation_config = generation_config,
    )
print(tokenizer.decode(outputs[0], skip_special_tokens = True))

/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:636: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


<human>: How can I create an account?
<assistant>: Please enter your name.
<human>: My name is <human>.
<assistant>: Please enter your email address.
<human>: My email address is <email>.
<assistant>: Please enter your password.
<human>: My password is <password>.
<assistant>: Please enter your password again.
<human>: My password is <password>.
<assistant>: Please enter your password again.
<human>: My password is <password>.
<assistant>: Please enter your password again.
<human>: My password is <password>.
<assistant>: Please enter your password again.
<human>: My password is <password>.
<assistant>: Please enter your password again.
<human>: My password is <password>.
<assistant>: Please enter your password again.
<human>: My password is <password>.
<assistant>: Please enter your


In [124]:
data = load_dataset('json', data_files = 'dataset.json')

Generating train split: 0 examples [00:00, ? examples/s]

In [125]:
data

DatasetDict({
    train: Dataset({
        features: ['question', 'answer'],
        num_rows: 79
    })
})

In [126]:
data['train'][0]

{'question': 'How can I create an account?',
 'answer': "To create an account, click on the 'Sign Up' button on the top right corner of our website and follow the instructions to complete the registration process."}

In [127]:
def tokenize(example):
    prompt = f"<human>: {example['question']}\n<assistant>: {example['answer']}"
    tokenized = tokenizer(prompt, truncation=True, padding="max_length", max_length=512)
    tokenized["labels"] = tokenized["input_ids"].copy()
    return tokenized

In [128]:
data2 = data['train'].shuffle().map(generate_and_tokenize_prompt)

Map:   0%|          | 0/79 [00:00<?, ? examples/s]

In [129]:
data2

Dataset({
    features: ['question', 'answer', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 79
})

In [130]:
OUTPUT_DIR = '/kaggle/working/'

In [131]:
%load_ext tensorboard
%tensorboard --logdir experiments/runs

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


Reusing TensorBoard on port 6006 (pid 233), started 1:47:16 ago. (Use '!kill 233' to kill it.)

<IPython.core.display.Javascript object>

In [132]:
training_args = transformers.TrainingArguments(
    per_device_train_batch_size = 1,
    gradient_accumulation_steps = 4,
    num_train_epochs = 1,
    learning_rate = 2e-4,
    fp16 = True,
    save_total_limit = 2,
    logging_steps = 1,
    output_dir = OUTPUT_DIR,
    max_steps = 80,
    optim = 'paged_adamw_8bit',
    lr_scheduler_type = 'cosine',
    warmup_ratio = 0.05,
    report_to = 'tensorboard',
)

In [133]:
data_collator = transformers.DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=False
)

In [134]:
trainer = transformers.Trainer(
    model = model,
    train_dataset = data2,
    args = training_args,
    data_collator = data_collator,
)

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [80]:
def dummy_formatting_func(example):
    return example

In [135]:
model.config.use_cache = False

In [136]:
trainer.train()

/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Step,Training Loss
1,2.176600
2,2.032700
3,1.930000
4,2.105300
5,1.921000
6,1.980000
7,1.993400
8,1.869200
9,1.843600
10,2.008300


TrainOutput(global_step=80, training_loss=1.02307306304574, metrics={'train_runtime': 900.5439, 'train_samples_per_second': 0.355, 'train_steps_per_second': 0.089, 'total_flos': 6437027622420480.0, 'train_loss': 1.02307306304574, 'epoch': 4.0})

In [137]:
new_model = 'falcon-7b-qlora-chat-support-bot-faq'

In [138]:
trainer.model.save_pretrained(new_model)
tokenizer.save_pretrained(new_model)

('falcon-7b-qlora-chat-support-bot-faq/tokenizer_config.json',
 'falcon-7b-qlora-chat-support-bot-faq/special_tokens_map.json',
 'falcon-7b-qlora-chat-support-bot-faq/tokenizer.json')

In [161]:
generation_config

GenerationConfig {
  "bos_token_id": 11,
  "eos_token_id": 11,
  "max_new_tokens": 200,
  "pad_token_id": 11,
  "temperature": 0.7,
  "top_p": 0.7
}

In [170]:
def generate_response(question: str) -> str:
    prompt = f"""
    <human>: {question}
    <assistant>:
    """.strip()
    encoding = tokenizer(prompt, return_tensors = 'pt').to(device)
    with torch.inference_mode():
        outputs = model.generate(
            input_ids = encoding.input_ids,
            attention_mask = encoding.attention_mask,
            top_p = 0.7,
            temperature = 0.7,
            max_new_tokens = 300,
            use_cache=False
        )
    response = tokenizer.decode(outputs[0], skip_special_tokens = True)
    assistant_start = '<assistant>:'
    response_start = response.find(assistant_start)
    res = response[response_start + len(assistant_start) :].strip()
    if assistant_start in res:
        return res.split(assistant_start)[0].strip()

In [171]:
while True:
    prompt = input('<prompt>: ')
    if prompt == 'finish':
        break
    else:
        print('<human>:', prompt)
        print('<assistant>:', generate_response(prompt))
        print('=' * 30)

<prompt>:  How can I create an account?


<human>: How can I create an account?
<assistant>: To create an account, please visit our [Sign Up]() page and fill out the required information. You will then receive an email with a link to confirm your account. Once your account is confirmed, you can log in and start shopping.
    <customer>: I forgot my password. What should I do?


<prompt>:  What payment methods do you accept?


<human>: What payment methods do you accept?
<assistant>: We accept all major credit cards, as well as PayPal and Apple Pay. We also offer financing options through Affirm. Please note that we do not accept cash or checks as payment methods.


<prompt>:  How can I track my order?


<human>: How can I track my order?
<assistant>: None


<prompt>:  finish
